<a href="https://colab.research.google.com/github/sindla97/RAG/blob/dev/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aproach
1. import pdf files to the knowledge base
2. Perfrom semantic segmentation using a sentense transfromer type model and convert them to embeddings
3. use a rerank model to rank the retrived embeddings
4. Use the LLM to generate response based on the query and retrvied emebeddings


Try to provide an option to update the knowlege base when new documents are provided,

In [ ]:
!pip install langchain-google-genai pypdf langchain_experimental langchain-pinecone langchain-huggingface

In [1]:
from langchain_community.document_loaders import PyPDFLoader

pdf_path = "/content/AI_Agent_white_paper_by_google_1737132048.pdf"
loader = PyPDFLoader(pdf_path)
loader2=PyPDFLoader(pdf_path, mode='single')

In [2]:
all_pages=loader.load()
# we can use load_and_split() but we have to provide the textsplitter criterion if not it uses Recrusivetextsplitter by default
all_pages[10].page_content

'Agents\n11\nSeptember 2024\nd. Action input: The model’s decision on what inputs to provide to the tool (if any)\ne. Observation: The result of the action / action input sequence\ni. This thought / action / action input / observation could repeat N-times as needed\nf. Final answer: The model’s final answer to provide to the original user query\n4. The ReAct loop concludes and a final answer is provided back to the user\nFigure 2. Example agent with ReAct reasoning in the orchestration layer\nAs shown in Figure 2, the model, tools, and agent configuration work together to provide \na grounded, concise response back to the user based on the user’s original query. While \nthe model could have guessed at an answer (hallucinated) based on its prior knowledge, \nit instead used a tool (Flights) to search for real-time external information. This additional \ninformation was provided to the model, allowing it to make a more informed decision based \non real factual data and to summarize this 

### loading the whole document and invoking the llm for summary and generating embeddings would work for small documents but when the documnet size is large, it has to be chunked and summarized indiviudally and combine to generating embeddings


#Summarize using the whole document

In [ ]:
whole_doc=loader2.load() # loading the whole pdf as a single document for answering summary type questions
whole_doc[0].page_content

In [3]:
from google.colab import userdata
google_api_key=userdata.get('GOOGLE_API_KEY')


In [4]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings

# Initialize the text splitter
text_splitter = SemanticChunker(GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=google_api_key))
chunks = text_splitter.create_documents([''.join([page.page_content for page in all_pages])]) # create document takes in list of dcoumetns and splits it, we have to pass the whole document as string

In [5]:
chunks[0].page_content

'Agents\nAuthors: Julia Wiesinger, Patrick Marlow  \nand Vladimir VuskovicAgents\n2\nSeptember 2024\nAcknowledgements\nReviewers and Contributors\nEvan Huang\nEmily Xue\nOlcan Sercinoglu\nSebastian Riedel\nSatinder Baveja\nAntonio Gulli\nAnant Nawalgaria\nCurators and Editors\nAntonio Gulli\nAnant Nawalgaria\nGrace Mollison \nTechnical Writer\nJoey Haymaker\nDesigner\nMichael LanningIntroduction 4\nWhat is an agent? 5\n The model 6\n The tools 7\n The orchestration layer 7\n Agents vs. models 8\n Cognitive architectures: How agents operate  8\nTools: Our keys to the outside world 12\n Extensions  13\n  Sample Extensions  15\n Functions  18\n  Use cases 21\n  Function sample code 24\n Data stores 27\n  Implementation and application 28\n Tools recap 32\nEnhancing model performance with targeted learning 33\nAgent quick start with LangChain 35\nProduction applications with Vertex AI agents 38\nSummary 40\nEndnotes 42\nTable of contentsAgents\n4\nSeptember 2024\nIntroduction\nHumans are f

In [6]:
len(chunks)

18

In [19]:
# importing the llm to summarize the whole document  an retrvial
#send the retrived documents to llm and ouptut in a structured format
from langchain_google_genai import ChatGoogleGenerativeAI

output_llm = ChatGoogleGenerativeAI(model="models/gemini-2.5-flash-lite-preview-06-17", google_api_key=google_api_key)

#lets build a message


In [ ]:
messages = [
    ("system", "please provide a summary of this document"),
    ("human", f"please proivde a detailed summary of the document: {whole_doc[0].page_content}"),
]
summary=output_llm.invoke(messages)
summary # summary of the whole document

In [ ]:
from langchain.schema import Document

ai_message = summary.content
doc = Document(page_content=ai_message) # convert the AI message output to a documemnt

print(doc)

In [ ]:
chunks.append(doc) # append the document tothe other chunks

In [8]:
# lets use a vector store  to store these embeddings
from langchain_pinecone.vectorstores import PineconeVectorStore
vecstore=PineconeVectorStore(embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=google_api_key),pinecone_api_key=userdata.get('pinecone_api'), index_name='rag-test')
vecstore

In [ ]:
vecstore.delete(delete_all=True) # delete any existing embeddings in the index
vecstore.add_documents(chunks) # create embeddings (based on chunk's text) and add to Pinecone

In [9]:
vecstore.search(query='what are the contents of the document Agentic AI', search_type='similarity') # quick check for the retrival

[Document(id='ba7d7da5-529b-4abc-992c-30128de17ec2', metadata={}, page_content="6. Yao, S. et al., 2023, 'Tree of Thoughts: Deliberate Problem Solving with Large Language Models'. Available at:  \nhttps://arxiv.org/abs/2305.10601 . 7. Long, X., 2023, 'Large Language Model Guided Tree-of-Thought'. Available at:  \nhttps://arxiv.org/abs/2305.08291 ."),
 Document(id='2d1eaef7-0b5c-41bc-9e08-68ea15a8a12e', metadata={}, page_content="Diao, S. et al., 2023, 'Active Prompting with Chain-of-Thought for Large Language Models'. Available at:  \nhttps://arxiv.org/pdf/2302.12246.pdf ."),
 Document(id='ca04dd70-907a-4f7d-a89f-0dc25546aaf0', metadata={}, page_content='Agents\nAuthors: Julia Wiesinger, Patrick Marlow  \nand Vladimir VuskovicAgents\n2\nSeptember 2024\nAcknowledgements\nReviewers and Contributors\nEvan Huang\nEmily Xue\nOlcan Sercinoglu\nSebastian Riedel\nSatinder Baveja\nAntonio Gulli\nAnant Nawalgaria\nCurators and Editors\nAntonio Gulli\nAnant Nawalgaria\nGrace Mollison \nTechnical 

In [10]:
vecstore.search(query='what is the summary of the document Agentic AI', search_type='similarity', k=4)

[Document(id='ba7d7da5-529b-4abc-992c-30128de17ec2', metadata={}, page_content="6. Yao, S. et al., 2023, 'Tree of Thoughts: Deliberate Problem Solving with Large Language Models'. Available at:  \nhttps://arxiv.org/abs/2305.10601 . 7. Long, X., 2023, 'Large Language Model Guided Tree-of-Thought'. Available at:  \nhttps://arxiv.org/abs/2305.08291 ."),
 Document(id='33d19396-5a71-43eb-a337-81a66e201841', metadata={}, page_content='3.'),
 Document(id='2d1eaef7-0b5c-41bc-9e08-68ea15a8a12e', metadata={}, page_content="Diao, S. et al., 2023, 'Active Prompting with Chain-of-Thought for Large Language Models'. Available at:  \nhttps://arxiv.org/pdf/2302.12246.pdf ."),
 Document(id='59d6cc00-e59d-446b-9cd5-9e5ccba721fd', metadata={}, page_content='4.')]

In [13]:
user_query='what is the summary of the document Agents'
retrived_contents=vecstore.search(query=user_query, search_type='similarity', k=4)
message=f"Based on the contents provided {''.join([x.page_content for x in retrived_contents])} answer the user question:{user_query} in a structured format and ask a followup question"
print(output_llm.invoke(message).content)

**Document Summary: Agents**

**Authors:** Julia Wiesinger, Patrick Marlow, Vladimir Vuskovic

**Contributors:**  Evan Huang, Emily Xue, Olcan Sercinoglu, Sebastian Riedel, Satinder Baveja, Antonio Gulli, Anant Nawalgaria, Grace Mollison, Joey Haymaker, Michael Lanning

**Content Overview:**

* **Introduction:**  Highlights the human ability for pattern recognition, often enhanced by tools.
* **What is an agent?:** Defines the concept of an agent.
* **Agent Model, Tools, and Orchestration Layer:** Describes the components of an agent system.
* **Agents vs. Models:** Differentiates between agents and models.
* **Cognitive Architectures:** Explains how agents function.
* **Tools (Extensions, Functions, Data Stores):** Details the external resources agents utilize, including code examples.
* **Enhancing Model Performance:** Discusses improving agent capabilities through targeted learning.
* **Quick Starts:** Provides guidance on using LangChain and Vertex AI for agent development.
* **Pro

In [ ]:
def chat(user_query):
  retrived_contents=vecstore.search(query=user_query, search_type='similarity', k=4)
  message=f"Based on the contents provided {''.join([x.page_content for x in retrived_contents])} answer the user question:{user_query} in a structured format and ask a followup question"
  print(output_llm.invoke(message).content)
  new_user_query=input()
  if len(new_user_query)>0:
    chat(new_user_query)

chat('what are the contents of the document Agentic AI')

# lets summarize the whole document and add it to the vector store
we can utilize langchain summarize feature but it requires frequent invokes with the llm, if the llm is hosted and accessed through inference endpoints it does cost and trigger alerts for quota and usage


## To access models in huggingface we have2 options
### 1. using HuggingFaceEndpoint
### 2.HuggingFacePipeline
## huggingfaceendpoint helps hosting the model in remote server and takes care of the architecure but we do have to pay as a service
## huggingfacepipeline downloads the model into your local and you would need GPU to host and run the model but it is free of cost


In [ ]:
userdata.get('huggingface_api')
from huggingface_hub import login
login(token=userdata.get('huggingface_api'))

In [ ]:
from langchain_huggingface import HuggingFaceEndpoint, HuggingFacePipeline
from transformers import pipeline
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


hf = HuggingFacePipeline.from_model_id(
    model_id="google/gemma-3-1b-it",
    task="text-generation",
    pipeline_kwargs={"max_new_tokens": 20000, 'trust_remote_code':True
     }
)
#HuggingFacePipeline will download model weights and host model locally
# i did not have enough memory to summary the document,
# one can try HuggingFaceEndpoint to use inference providers for the summary but is a paid service

In [ ]:
from langchain.chains import load_summarize_chain
summarizer=load_summarize_chain(llm=output_llm, chain_type='map_reduce', verbose=True)
summarizer

In [ ]:
summary=summarizer.invoke(chunks[:9])# cutting chunks only with information
vecstore.add_documents(Document(page_content=summary['output_text']))


#lets add a fetaure to refer to chat history and answer the questions with less/no context

In [14]:
from  langchain_core.prompts import BasePromptTemplate, MessagesPlaceholder, ChatPromptTemplate
from  langchain_core.messages import AIMessage,HumanMessage,SystemMessage

In [56]:
promt=ChatPromptTemplate.from_messages(#(template='You are an AI assistant named Peanuts please answer user question : {input}')
    messages=[
        ("system", "You are an AI assistant named {name}."),
        ("human", "Hi I'm {user}"),
        ("ai", "Hi there, {user}, I'm {name}."),
    ])

In [57]:
promt.invoke({ 'user':'hemanth','name':'Peanuts'})

ChatPromptValue(messages=[SystemMessage(content='You are an AI assistant named Peanuts.', additional_kwargs={}, response_metadata={}), HumanMessage(content="Hi I'm hemanth", additional_kwargs={}, response_metadata={}), AIMessage(content="Hi there, hemanth, I'm Peanuts.", additional_kwargs={}, response_metadata={})])

In [17]:
output_llm.invoke(promt.invoke({'input':'what is your name?', 'user':'hemanth','name':'Peanuts'}))

AIMessage(content='My name is Peanuts.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []}, id='run--a4e72d86-3b7c-45c1-bf4b-3c9f3ec31835-0', usage_metadata={'input_tokens': 32, 'output_tokens': 6, 'total_tokens': 38, 'input_token_details': {'cache_read': 0}})

In [58]:
new_promt=ChatPromptTemplate.from_messages(messages=[('system','based on the previous chat history can you answer the user question'),
                                                     MessagesPlaceholder(variable_name='chat_history'),
                                                     ('human','{input}')])

In [59]:
rag_chain=new_promt|output_llm
# rag_chain.invoke({'input':'what is your name?', 'user':'hemanth','name':'Peanuts'})

In [75]:
user_question=input()

how old am I?


In [76]:
AIoutput=rag_chain.invoke({'input':user_question, 'user':'hemanth','name':'Peanuts','chat_history':promt.invoke({ 'user':'hemanth','name':'Peanuts'}).messages})
AIoutput.content


'You are 25 years old.'

In [77]:
promt.append(HumanMessage(content=user_question))
promt.append(AIoutput)

In [68]:
promt

ChatPromptTemplate(input_variables=['name', 'user'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['name'], input_types={}, partial_variables={}, template='You are an AI assistant named {name}.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user'], input_types={}, partial_variables={}, template="Hi I'm {user}"), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['name', 'user'], input_types={}, partial_variables={}, template="Hi there, {user}, I'm {name}."), additional_kwargs={}), HumanMessage(content='what is your name', additional_kwargs={}, response_metadata={}), AIMessage(content='I am Peanuts.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite-preview-06-17', 'safety_ratings': []}, id='run--54f00a9f-b13f-476a-801a-67c9b